# Punto 6. Workshop 6

In [39]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx
import shapely

plt.rcParams['figure.figsize'] = (10, 8)

root_folder= r"/home/DATA"


SHAPE_MUN = root_folder+r"/municipios_colombia.shp"
EVA_CSV   = root_folder+'/Evaluaciones_Agropecuarias_Municipales_EVA.csv'
EST_CSV   = root_folder+'/Catalogo_Estaciones_IDEAM.csv'
SHAPE_VIAS = root_folder+r"/vias_colombia_V2.shp"

In [ ]:
gdf_vias = gpd.read_file(SHAPE_VIAS).to_crs(epsg=9377)

In [ ]:
# Municipalities → GeoDataFrame 
gdf_mun = gpd.read_file(SHAPE_MUN).to_crs(epsg=9377)

# EVA table, rename code column
eva = (pd.read_csv(EVA_CSV)
         .rename(columns={"CÓD. MUN.": "cod_mun"}))
eva["CULTIVO"] = eva["CULTIVO"].astype(str)      # ensure string

# IDEAM catalogue → GeoDataFrame (clean "(lat, lon)")
df_est = pd.read_csv(EST_CSV)
coords = (df_est["Ubicación"]
            .str.strip("()")
            .str.split(",", expand=True)
            .astype(float))
df_est["lat"], df_est["lon"] = coords[0], coords[1]

gdf_est = gpd.GeoDataFrame(
    df_est,
    geometry=gpd.points_from_xy(df_est["lon"], df_est["lat"]),
    crs="EPSG:4326"
).to_crs(epsg=9377)

print(f"{len(gdf_mun):,} municipalities • {len(gdf_est):,} stations • "
      f"{len(eva):,} EVA rows")


#Homogenise municipalities codes
gdf_mun["MPIO_CCDGO"] = gdf_mun["MPIO_CCDGO"].astype(str).str.zfill(5)
eva["cod_mun"]        = eva["cod_mun"].astype(str).str.zfill(5)

# 6.2 Pasos de análisis básicos - Nivel 1: Selección municipal

In [ ]:
gdf_mun.head()

gdf_mun['AREA'] = gdf_mun.area

print (gdf_mun)

In [ ]:
# Paso 1: Convertir la columna a tipo numérico, manejando posibles errores

gdf_mun["DPTO_CCDGO_NUM"] = pd.to_numeric(gdf_mun["DPTO_CCDGO"], errors='coerce')

# Paso 2: Filtrar usando la nueva columna numérica
# Aquí es importante asegurarte de que 15 sea el tipo correcto (int) para la comparación
Boyaca = gdf_mun[gdf_mun["DPTO_CCDGO_NUM"] == 15].copy()

# Ahora, verifica si Boyaca está vacío antes de continuar
if Boyaca.empty:
    print("¡Advertencia! No se encontraron municipios para el código de departamento 15.")
    print("Revisa los valores únicos en la columna 'DPTO_CCDGO' de gdf_mun para depurar.")
    # Opcional: mostrar los primeros valores para inspección
    # print(gdf_mun["DPTO_CCDGO"].head())
    # print(gdf_mun["DPTO_CCDGO"].dtype)
else:
    print(f"Se encontraron {len(Boyaca)} municipios para el código 15.")
    boyaca_union_geometry = Boyaca.geometry.union_all()
    Boyaca_vias = gdf_vias[gdf_vias.intersects(boyaca_union_geometry)]
    
    Boyaca_vias =  gpd.clip(Boyaca_vias,Boyaca)
    print("\n--- Vías encontradas en Boyacá (todas las municipalidades) ---")
    print(Boyaca_vias)

In [ ]:
Boyaca.head()

In [ ]:
Boyaca_poligono = Boyaca.dissolve(by = 'DPTO_CCDGO_NUM')

Boyaca_poligono.head()
print("\nGeoDataFrame Boyaca_poligono (después de dissolve):\n", Boyaca_poligono)

In [ ]:
# --- Plotting (Visualización) ---
print("\n--- Generando el gráfico ---")
fig, ax = plt.subplots(1, 1, figsize=(10, 10)) # Crear figura y ejes explícitamente para mejor control

# Plotear el polígono de Boyacá (disuelto)
Boyaca_poligono.plot(ax=ax, color="lightgreen", edgecolor="black", alpha=0.7)

# Plotear las vías de Boyacá encima del polígono
if not Boyaca_vias.empty: # Solo plotea si hay vías
    Boyaca_vias.plot(ax=ax, color="red", linewidth=2)
    print("Vías ploteadas con éxito.")
else:
    print("Advertencia: Boyaca_vias está vacío, no se plotearon vías.")

ax.set_title("Vías sobre el Polígono de Boyacá")
ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
Boyaca = gdf_mun[gdf_mun["DPTO_CCDGO"].str.upper().str.contains("15")].copy()
Boyaca_bool = gdf_est.intersects(Boyaca.geometry.iloc[0])
Boyaca_stations = gdf_est[Boyaca_bool]
print(Boyaca_stations)


In [ ]:
Boyaca.head()


In [ ]:
ax = Boyaca.plot(color="none", edgecolor="grey", figsize=(8, 8))
Boyaca_stations.plot(ax=ax, color="red", markersize=6)
plt.title("IDEAM stations (projected) over Depto 18")
plt.show()

In [ ]:
ax = Boyaca_poligono.plot(color="none", edgecolor="grey", figsize=(8, 8))
Boyaca_stations.plot(ax=ax, color="red", markersize=6)
plt.title("IDEAM stations (projected) over Depto 18")
plt.show()

In [ ]:
ax = Boyaca_poligono.plot(color="none", edgecolor="grey", figsize=(8, 8))
Boyaca_vias.plot(ax=ax, color="red", markersize=6)
plt.title("IDEAM stations (projected) over Depto 18")
plt.show()

In [ ]:

Boyaca_vias_copy = Boyaca_vias.copy()

# Create a 10 km buffer around each station
Boyaca_vias_copy['buffer_10km'] = Boyaca_vias_copy.geometry.buffer(10000)

# Visualise one example station buffer
ax = Boyaca_poligono.plot(color='none', edgecolor='lightgrey', figsize=(8, 8))

# Usa Boyaca_vias_copy para el buffer
gpd.GeoSeries(Boyaca_vias_copy['buffer_10km']).plot(ax=ax, facecolor='lightblue', edgecolor='blue', alpha=0.5)
Boyaca_vias_copy.plot(ax=ax, color='red', markersize=20) # También usa la copia aquí

plt.title("10 km Buffers around vias Boyaca")
plt.show()

In [ ]:
Boyaca_poligono.head()

In [ ]:
# 1. Spatial join: each station takes the attributes of the municipality it falls in
stations_with_mun = gpd.sjoin(
    Boyaca_stations,                     # left: point layer
    gdf_mun[["MPIO_CCDGO", "MPIO_CNMBR","MPIO_CCDGO", "geometry"]],  # right: polygon layer
    how="left",
    predicate="within"           # point inside polygon
)

# 2. Inspect the result
stations_with_mun[["Codigo", "MPIO_CNMBR", "MPIO_CCDGO"]].head()

In [ ]:
import numpy as np
import shapely

# Create a regular grid covering Colombia

# Bounds of the municipalities layer
bounds = gdf_mun.total_bounds
xmin, ymin, xmax, ymax = bounds

# Define grid resolution (example: 100 km x 100 km grid → 100_000 m)
res = 100000

# Build grid coordinates
cols = list(range(int(np.floor(xmin)), int(np.ceil(xmax + res)), res))
rows = list(range(int(np.floor(ymin)), int(np.ceil(ymax + res)), res))
rows.reverse()

# Build polygons for each grid cell
polygons = []
for x in cols:
    for y in rows:
        polygons.append(
            shapely.Polygon([
                (x, y),
                (x + res, y),
                (x + res, y - res),
                (x, y - res)
            ])
        )

# Create grid GeoDataFrame
grid = gpd.GeoDataFrame({'geometry': polygons}, crs=gdf_mun.crs)

# Remove cells that fall outside the total extent (intersect municipalities only)
sel = grid.intersects(shapely.box(*bounds))
grid = grid[sel]

# Add unique ID
grid['id'] = grid.index

grid.head()

In [ ]:
eva.head()

In [ ]:
gdf_mun["MPIO_CCDGO"] = gdf_mun["MPIO_CCDGO"].astype(str).str.zfill(5)
eva["cod_mun"]        = eva["cod_mun"].astype(str).str.zfill(5)

In [ ]:
# Calculate polygon area in square metres
gdf_vias['longitud'] = gdf_vias.length

gdf_vias.head()

In [ ]:
#vias_with_mun = gpd.sjoin(
 #   gdf_mun,                     # left: point layer
  #  gdf_vias[["longitud", "geometry"]],  # right: polygon layer
   # how="left",
    #predicate="intersects"           # point inside polygon
#)

# 2. Inspect the result
#vias_with_mun.head()


In [ ]:
#gdf_interseccion = gdf_mun.overlay(vias_with_mun, how='intersection')
#gdf_interseccion.head()

In [ ]:
#mun_grid = gdf_interseccion.overlay(grid)

# Keep only needed columns
#mun_grid = mun_grid[['MPIO_CNMBR', 'longitud', 'geometry']]

# Compute the area of each piece (intersection area)
#mun_grid['area_sub'] = mun_grid.area

#mun_grid.head()

In [ ]:
#ax = mun_grid.plot(
 #   column='area_sub',
  #  edgecolor='black',
   # legend=True,
    #cmap='Reds',
   # figsize=(10, 10),
    #vmin=100000000,
   # vmax=10000000000
#)

# Title
#plt.title("Effect of .overlay:")
#plt.show()

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

In [ ]:
# Calculate polygon area in square metres
Boyaca_vias_copy['longitud'] = Boyaca_vias_copy.length

Boyaca_vias_copy.head()

In [ ]:
Boyaca.head()

In [ ]:
# Realizar la intersección
union_gdf = gpd.overlay(Boyaca, Boyaca_vias_copy,keep_geom_type=False)
union_gdf.tail()

In [ ]:
#Para verificar si quedo bien el overlay
#Tunja_vias = union_gdf[union_gdf['MPIO_CCDGO']=='00332']
#Tunja_vias.head()

In [ ]:
# Calculate polygon area in square metres
union_gdf['longitud_nueva'] = union_gdf.length

union_gdf.head()

In [ ]:
#para verificar si efectivamente se realizaron los ajustes

#print("antes",Tunja_vias.head(1))
#Tunja_vias["longitufd_guican"]=Tunja_vias.length
#print("despues",Tunja_vias.head(1))

In [ ]:
#ax = Tunja_vias.plot(
#    column='longitud',
#    edgecolor='black',
#    legend=True,
#    cmap='Reds',
#    figsize=(10, 10),
#    vmin=100000000,
#    vmax=10000000000
#)

# Title
#plt.title("Effect of .overlay:")
#plt.show()

In [ ]:
valores_unicos = union_gdf['Codigo_Mun'].unique()

# Imprimir los valores únicos
print("Valores únicos de la columna 'Codigo_Mun':")
print(valores_unicos)

In [ ]:
union_gdf['suma_vias_por_municipio'] = union_gdf.groupby('Codigo_Mun')['longitud_nueva'].cumsum()

union_gdf.head()

In [ ]:
union_gdf_geometria = union_gdf.geometry.geom_type.unique()
grid_geometria = grid.geometry.geom_type.unique()

print (union_gdf_geometria, 'union_gdf_geometria')

print (grid_geometria, 'grid_geometria')

In [ ]:
# 1. Spatial join: each station takes the attributes of the municipality it falls in
vias_por_mun = gpd.sjoin(
    union_gdf,                     # left: point layer
    gdf_mun[["MPIO_CCDGO", "MPIO_CNMBR","MPIO_CCDGO", "geometry"]],  # right: polygon layer
    how="left",
    predicate="within"           # point inside polygon
)

# 2. Inspect the result
vias_por_mun.tail()

In [ ]:
municipios_geom = gdf_mun[["MPIO_CCDGO", "geometry"]].rename(columns={"MPIO_CCDGO": "MPIO_CCDGO_left"})

# Sumar las longitudes por municipio y luego unir con la geometría del municipio
sumatoria_munic_df = vias_por_mun.groupby('MPIO_CCDGO_left')['longitud_nueva'].sum().reset_index()

# Ahora, convertir sumatoria_munic_df de nuevo a un GeoDataFrame, uniendo con las geometrías de los municipios.
# La clave para el join es 'MPIO_CCDGO_left' (o el nombre de la columna que usas para identificar el municipio).
sumatoria_munic = pd.merge(sumatoria_munic_df, municipios_geom, on='MPIO_CCDGO_left', how='left')

# Convertir el DataFrame resultante a un GeoDataFrame
sumatoria_munic = gpd.GeoDataFrame(sumatoria_munic, geometry='geometry')

print(sumatoria_munic)



In [ ]:
print(mun_grid.columns)

In [ ]:
# Ahora sí, sumatoria_munic es un GeoDataFrame y puedes usar overlay
mun_grid = sumatoria_munic.overlay(grid, how='intersection')

print('mun_grid', mun_grid)

# Keep only needed columns
# Es posible que los nombres de las columnas cambien ligeramente después del overlay,
# especialmente si 'MPIO_CNMBR_left' no se propaga directamente o si se generan duplicados.
# Revisa las columnas de 'mun_grid' después de la ejecución.
mun_grid = mun_grid[['MPIO_CNMBR', 'longitud_nueva', 'geometry']] # Asumo que 'longitud' es la suma de las vías

# Compute the area of each piece (intersection area)
mun_grid['area_sub'] = mun_grid.area

mun_grid.head()

In [ ]:
sumatoria_munic = vias_por_mun.groupby(['MPIO_CCDGO_left', 'AREA'])['longitud'].sum().reset_index()

print (sumatoria_munic)

In [ ]:
mun_grid = sumatoria_munic.overlay(grid, how='intersection')

print ('mun_grid', mun_grid)

#Keep only needed columns
mun_grid = mun_grid[['MPIO_CNMBR_left', 'suma_vias_por_municipio', 'geometry']]

# Compute the area of each piece (intersection area)
mun_grid['area_sub'] = mun_grid.area

mun_grid.head()